In [7]:
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import os

In [50]:
"""
Input for the code:
    $ A fasta file containing your DNA sequence that you want to edit with gRNAs (eg. pXW487, in fasta format)
    $ A text file containing:
         gRNA sequences
         PAM sequences

Output:
    $ Oligos that make the donor DNA to replace gRNA to a common placeholder
"""

SEQ_FILENAME = "pXW487.fasta"
gRNA_FILENAME = "grna_pam_strand.txt"
Oligo_FILENAME = "Donor_oligos.txt"

desktop = os.path.expanduser("~/Desktop")

seq_file_path = desktop + '/' + SEQ_FILENAME
gRNA_file_path = desktop + '/' + gRNA_FILENAME
oligo_file_path = desktop + '/' + Oligo_FILENAME

In [51]:
# Load the fasta sequence from the file
def load_fasta_sequence(file_path):
    with open(file_path, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            return record.seq

In [52]:

my_sequence = load_fasta_sequence(seq_file_path)

gRNA_df = pd.read_csv(gRNA_file_path, sep='\t')
gRNA_df

,SNP,gRNA,PAM,PAM_pos,strand
0,rs10774035,CCCCTTGCAACATCTCAAAG,CGG,82431,-
1,rs10774036,CAATTTTTGGTGAGCGGTTA,TGG,100699,-
2,rs10744560,CTCAGCACGATGAGTGAGAC,TGG,100850,-
3,rs12311439,ACTCTGCTGTGCCGCAGAGC,TGG,108536,+
4,rs1024582,GTATTATTTGGTTGTTCACG,GGG,116001,+
5,rs4298967,AGGCTGAACGGAGCTAGTGA,GGG,121942,+


In [53]:
placeholder = "CCTCTTAGACGAAGGAAGTGTGG"
placeholder_rc = Seq(placeholder).reverse_complement()

In [54]:
gRNA_df["gRNA+PAM"] = gRNA_df["gRNA"] + gRNA_df["PAM"]

In [55]:
# Lists to store the two DNA sequences for each SNP
Fwd_oligos = []
Rev_oligos = []

for index, row in gRNA_df.iterrows():
    gRNA_PAM = Seq((row["gRNA"] + row["PAM"]).replace(" ", ""))

    # Find the position of gRNA_PAM in  my_sequence
    gRNA_pam_pos = my_sequence.find(gRNA_PAM)
    gRNA_pam_pos_rc = my_sequence.find(gRNA_PAM.reverse_complement())

    if gRNA_pam_pos != -1:
        # Extract the first DNA sequence (2a)
        sequence_2a = my_sequence[gRNA_pam_pos - 37: gRNA_pam_pos] + placeholder
        Fwd_oligos.append(str(sequence_2a))

        # Extract the second DNA sequence (2b)
        downstream_start = gRNA_pam_pos + len(gRNA_PAM)
        sequence_2b = Seq(my_sequence[downstream_start: downstream_start + 37]).reverse_complement() + placeholder_rc
        Rev_oligos.append(str(sequence_2b))

    elif gRNA_pam_pos_rc != -1:
        # Extract the first DNA sequence (2a)
        sequence_2a = my_sequence[gRNA_pam_pos_rc - 37: gRNA_pam_pos_rc] + placeholder
        Fwd_oligos.append(str(sequence_2a))

        # Extract the second DNA sequence (2b)
        downstream_start = gRNA_pam_pos_rc + len(gRNA_PAM)
        sequence_2b = Seq(my_sequence[downstream_start: downstream_start + 37]).reverse_complement() + placeholder_rc
        Rev_oligos.append(str(sequence_2b))

    else:
        Fwd_oligos.append("Not found")
        Rev_oligos.append("Not found")

# Add the sequences as new columns in the DataFrame
gRNA_df["Fwd_primers"] = Fwd_oligos
gRNA_df["Rev_primers"] = Rev_oligos

# Print the updated DataFrame
gRNA_df.to_csv(oligo_file_path,sep="\t")
print("Data saved to", oligo_file_path)